Calculateur d'itinéraire vélo à apprentissage supervisé
=============================

(Toute ma gratitude à qui trouve un nom plus vendeur)

Bienvenue sur la page de test ! Ceci est un notebook jupyter. Il y a des blocs de simple texte et des blocs de code. Pour tester le programme, faire shift+entrée sur les blocs de code les uns après les autres en suivant les instructions. Le * qui appraît à côté de « Entrée » signifie que L'ordinateur réfléchit. Attendez qu'un numéro s'affiche pour passer à la suite.
 Vous pouvez modifier les arguments des fonctions. En fait vous pouvez faire ce que vous voulez : ceci est une copie qui sera immédiatement effacée.

Principe du programme
------------------------

Le fichier csv chemins.csv contient une liste d'itinéraires rentrés par des cyclistes. La petite IA programmée dans le module «apprentissage» va en déduire un coefficient de cyclabilité pour chaque tronçon de route. Et le calculateur d'itinéraires prendra cette cyclabilité en compte.

Chargement des données
----------------------------

Pour commencer on charge les bibliothèques.

In [1]:
#Pour permettre l'affichage interactif des figures (zoom notamment)
import matplotlib
matplotlib.use('nbagg')
from IPython.display import IFrame  # Affichage d’une page web dans le notebook
import osmnx as ox # Extraction des données openstreetmap
import os
import sys

In [4]:
#Modules de ce projet
from dijk.progs_python.params import TMP
from init_graphe import charge_graphe # initialisation du graphe
import apprentissage
import dijkstra
import chemins # lecture du csv, manipulation des chemins
import utils # les fonctions de base pour utiliser le logiciel

nouveau chargement de params.py


nouveau chargement de params.py


Chargement du xml dijk/progs_python/données/voies_et_nœuds.osm
fini

2021-09-04 15:18:59 Configured OSMnx 1.1.1
2021-09-04 15:18:59 HTTP response caching is on
2021-09-04 15:18:59 Configured OSMnx 1.1.1
2021-09-04 15:18:59 HTTP response caching is on


Chargement du graphe. L’argument facultatif bavard indique la quantité d’infos qui seront affichées

In [5]:
g = charge_graphe(bavard=1) 

2021-09-04 15:19:00 Converting node, edge, and graph-level attribute data types
Graphe en mémoire !2021-09-04 15:19:01 Loaded graph with 15478 nodes and 21550 edges from "dijk/progs_python/donne?es/-0.428543.2671-0.254143.3403.graphml"

Calcul de la version sans multiarêtes
Chargement du cache nœud_of_rue.
Chargement de la cyclabilité
Chargement de la cyclabilité
Ajout du nom des villes
Ajout de la liste des nœuds de chaque rue
Chargement de la liste des nœuds de chaque rue finie.
Chargement du graphe fini.



Test rapide
---------------

La fonction utils.itinéraire affiche l’itinéraire calculé entre deux adresses.

Le troisième argument (prérempli à 0.3) est la proportion de détour acceptée. En le mettant à 0 vous obtenez le plus court chemin. Si vous le mettez à 1 cela signifie que vous êtes prêt à faire un trajet deux fois plus long pour remplacer un trajet médiocre (référence : la rue Faisans) par une vraie piste cyclable (référence : celle du Fébus)

In [6]:
def itinéraire(départ, arrivée, p_détour, g, où_enregistrer=TMP, bavard=0):
    """ Version pour le notebook"""
    d = chemins.Étape(départ, g)
    a = chemins.Étape(arrivée, g)
    c = chemins.Chemin([d, a], p_détour, False)
    res = g.chemin_étapes_ensembles(c)
    graphe_c = g.multidigraphe.subgraph(res)
    carte = ox.plot_graph_folium(graphe_c, popup_attribute="name")
    nom = "tmp/résultat_itinéraire.html"
    carte.save(nom)
    print(nom)
    display(IFrame(nom, width=800, height=600))

In [7]:
os.getcwd()

'/home/moi/git/osm vélo/site_vélo'

In [8]:
itinéraire("lycée louis barthou", "parc lawrence", 0.3, g, bavard=3)

2021-09-04 15:19:08 Created edges GeoDataFrame from graph
tmp/résultat_itinéraire.html


In [9]:
# Sans détour à titre de comparaison
itinéraire("lycée louis barthou", "parc lawrence", 0, g, bavard=3)

2021-09-04 15:19:14 Created edges GeoDataFrame from graph
tmp/résultat_itinéraire.html


Pour jouer avec l’IA
------------------------

La commande suivante charge les chemins dont je dispose:

In [8]:
tous_les_chemins = chemins.chemins_of_csv(g)


 ['oui', '30', 'rue des Véroniques; rue Voltaire; rue Sambre et Meuse;cours Bosquet; place de la République']
Mis en cache : [459812758, 459812765, 459812764, 8943628592, 459812771] pour rue des veroniques
Mis en cache : [459278230, 459278232, 459278235, 459278234, 766490866] pour rue voltaire

 ['oui', '30', 'rue des réparatrices; avenue San Carlos;avenue des acacias;avenue de Montebello;avenue Bié-Moulié;rue Gutenberg']

 ['oui', '30', 'avenue du Loup;chemin des écoliers;rue Mozart;chemin Guilhem']

 ['oui', '30', 'Rond-Point Éric Tabarly; Chemin Labriart; Rue Cassiopée; Rue Blanqui; Rue Paul Bert; Rue Marx Dormoy; CSTJF']

 ['oui', '20', "rue d'étigny; place gramont; rue tran; place de la libération; rue gassiot; rue carnot; rue cazaubon norbert; boulevard d'alsace-lorraine; cours lyautey; avenue de saragosse; avenue de buros; 84 avenue de buros"]

 ['oui', '20', 'véloroute V81 (64320 mazères-lezons) ; véloroute V81 (64320 bizanos)  ; 13 rue de verdun (64320 bizanos) ; avenue alber

In [9]:
len(tous_les_chemins) # Pour voir combien il y en a

12

La fonction suivante permet de rajouter des chemins à la main :

In [10]:
def ajoute_chemin(étapes, AR=True, pourcentage_détour=30):
    """ Entrées : - étapes (str list) liste des adresses des étapes.
                  - AR (bool) indique si le retour est valable aussi
                  - pourcentage_détour (int) pourcentage de détour accepté par le cycliste qui propose cet iténéraire."""
    tous_les_chemins.append(chemins.Chemin.of_étapes(étapes, pourcentage_détour, AR, g))

Voici un exemple d'utilisation. Attention, ici on utilise un pourcentage de détour, au lieu d’une proportion. En pratique c’est 100 fois plus.

In [11]:
ajoute_chemin( ["lycée Louis Barthou", "rue Lamothe", "rue Jean Monnet", "rue Galos", "place de la république"], True, 20)

Mis en cache : [307639922, 6152162709] pour rue lamothe
Mis en cache : [296917292, 6515105577, 296917958, 6152162709] pour rue jean monnet
Mis en cache : [339675110, 913161306, 339675112] pour rue galos


In [12]:
len(tous_les_chemins)

13

Utilisez cette fonction pour rajouter autant de chemins que vous le souhaitez !

La fonction suivant élimine les chemins incorrects (notamment s’il y a une faute de frappe dans un des noms de rues)

In [13]:
tous_les_chemins = utils.cheminsValides(tous_les_chemins, g)

In [14]:
len(tous_les_chemins)

13

Pour réinitialiser l’IA:

In [15]:
g.réinitialise_cyclabilité()

Et pour la ré-entrainer avec tous les chemins :

In [16]:
apprentissage.n_lectures(10, g, tous_les_chemins, bavard=1)

Début de l'apprentissage. C'est parti pour 10 lectures.
 ----- Étape 0, 1555 arêtes modifiées -----
 ----- Étape 1, 1281 arêtes modifiées -----
 ----- Étape 2, 918 arêtes modifiées -----
 ----- Étape 3, 602 arêtes modifiées -----
 ----- Étape 4, 524 arêtes modifiées -----
 ----- Étape 5, 332 arêtes modifiées -----
 ----- Étape 6, 320 arêtes modifiées -----
 ----- Étape 7, 317 arêtes modifiées -----
 ----- Étape 8, 241 arêtes modifiées -----
 ----- Étape 9, 203 arêtes modifiées -----


Maintenant, vous pouvez relancer utils.itinéraire

In [26]:
itinéraire("lycée louis barthou", "rue castetnau", .3, g, bavard=1)

Mis en cache : [296917277, 339665895, 339665896, 340280212, 296917278, 340280208, 340280196, 339803903, 2970387517, 339803913, 339803911, 296917279] pour rue castetnau
2021-08-29 21:48:07 Created edges GeoDataFrame from graph
tmp/lycéelouisbarthouruecastetnau.html
